In [1]:
import os
from convnets import *
from __future__ import division, print_function

%matplotlib inline

from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
set_session(tf.Session(config=config))

Using TensorFlow backend.


# The Nature Conservancy: Fisheries Monitoring
The Conservancy has invited the Kaggle community to develop algorithms to automatically detect and classify species of tunas, sharks and more that fishing boats catch, which will accelerate the video review process.

We will apply transfer learning using the pretrained model VGG16, and replacing the top "classification" layers with:
- MLP augmented with dropout and batch-normalization
- fullcy convolutional classifier

## Getting Bottleneck features

In [2]:
train_path = './train/'
valid_path = './valid/'
test_path = './test/'
batch_size=32

In [3]:
model = get_VGG16()
train_bottleneck = get_batches(train_path,batch_size=batch_size,shuffle=False)
valid_bottleneck = get_batches(valid_path,batch_size=batch_size,shuffle=False)

Found 3022 images belonging to 8 classes.
Found 755 images belonging to 8 classes.


In [4]:
%%time
print('Getting bottleneck features for train')
bottleneck_features_train = model.predict_generator(train_bottleneck, train_bottleneck.samples // batch_size)
np.save(open('./initial_files/train_bottleneck.npy', 'wb'), bottleneck_features_train)

print('Getting bottleneck features for validation')
bottleneck_features_validation = model.predict_generator(valid_bottleneck, valid_bottleneck.samples // batch_size)
np.save(open('./initial_files/valid_bottleneck.npy', 'wb'), bottleneck_features_validation)

Getting bottleneck features for train
Getting bottleneck features for validation
CPU times: user 2min 18s, sys: 14.3 s, total: 2min 32s
Wall time: 1min 29s


## Fitting
#### 1 - MLP augmented with dropout and batch-normalization

In [5]:
mlp_vgg = top_model_vgg(X_shape=(14,14,512),n_classes=train_bottleneck.num_class,dense_neurons=512,lr=0.001,do=0.5)
mlp_vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
top_maxpooling (MaxPooling2D (None, 7, 7, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 512)         2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 512)         0         
_________________________________________________________________
top_flatten (Flatten)        (None, 25088)             0         
_________________________________________________________________
top_relu_1 (Dense)           (None, 512)               12845568  
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

In [6]:
mlp_vgg.fit(bottleneck_features_train,
          np_utils.to_categorical(train_bottleneck.classes[:len(bottleneck_features_train)]),
          epochs = 10,
          batch_size = batch_size,
          validation_data = (bottleneck_features_validation,
                             np_utils.to_categorical(valid_bottleneck.classes[:len(bottleneck_features_validation)])))

Train on 3008 samples, validate on 736 samples
Epoch 1/10
3008/3008 [==============================] - 4s - loss: 1.3110 - acc: 0.6273 - val_loss: 0.4399 - val_acc: 0.9035
Epoch 2/10
3008/3008 [==============================] - 3s - loss: 0.3857 - acc: 0.8920 - val_loss: 0.2462 - val_acc: 0.9402
Epoch 3/10
3008/3008 [==============================] - 3s - loss: 0.2320 - acc: 0.9312 - val_loss: 0.2233 - val_acc: 0.9592
Epoch 4/10
3008/3008 [==============================] - 3s - loss: 0.1514 - acc: 0.9558 - val_loss: 0.2035 - val_acc: 0.9606
Epoch 5/10
3008/3008 [==============================] - 3s - loss: 0.1020 - acc: 0.9678 - val_loss: 0.2308 - val_acc: 0.9606
Epoch 6/10
3008/3008 [==============================] - 3s - loss: 0.0705 - acc: 0.9751 - val_loss: 0.2722 - val_acc: 0.9565
Epoch 7/10
3008/3008 [==============================] - 3s - loss: 0.0806 - acc: 0.9761 - val_loss: 0.2463 - val_acc: 0.9633
Epoch 8/10
3008/3008 [==============================] - 3s - loss: 0.0775 - ac

#### 2 - Fully onvolutional layers

In [7]:
model_fc = heat_layers(X_shape=(14,14,512),n_classes=train_bottleneck.num_class,n_fm=256,do=0.5,lr=0.001)
model_fc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 14, 14, 512)       2048      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 256)       1179904   
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 256)       590080    
_________________________________________________________________
batch_normalization_6 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 256)       590080    
_________________________________________________________________
batch_normalization_7 (Batch (None, 14, 14, 256)       1024      
__________

In [8]:
model_fc.fit(bottleneck_features_train,
             np_utils.to_categorical(train_bottleneck.classes[:len(bottleneck_features_train)]),
             epochs = 10,
             batch_size = batch_size,
             validation_data = (bottleneck_features_validation,
                                np_utils.to_categorical(valid_bottleneck.classes[:len(bottleneck_features_validation)])))

Train on 3008 samples, validate on 736 samples
Epoch 1/10
3008/3008 [==============================] - 8s - loss: 0.9084 - acc: 0.7241 - val_loss: 1.4166 - val_acc: 0.6807
Epoch 2/10
3008/3008 [==============================] - 7s - loss: 0.3068 - acc: 0.9003 - val_loss: 0.5342 - val_acc: 0.8859
Epoch 3/10
3008/3008 [==============================] - 7s - loss: 0.1516 - acc: 0.9525 - val_loss: 0.3496 - val_acc: 0.9307
Epoch 4/10
3008/3008 [==============================] - 7s - loss: 0.1002 - acc: 0.9688 - val_loss: 0.3930 - val_acc: 0.9158
Epoch 5/10
3008/3008 [==============================] - 7s - loss: 0.1005 - acc: 0.9688 - val_loss: 0.3421 - val_acc: 0.9280
Epoch 6/10
3008/3008 [==============================] - 7s - loss: 0.0512 - acc: 0.9837 - val_loss: 0.3356 - val_acc: 0.9361
Epoch 7/10
3008/3008 [==============================] - 7s - loss: 0.0331 - acc: 0.9894 - val_loss: 0.3289 - val_acc: 0.9361
Epoch 8/10
3008/3008 [==============================] - 7s - loss: 0.0487 - ac

## Predict

In [ ]:
test = get_batches(test_path,batch_size=7,shuffle=False)

Found 13153 images belonging to 2 classes.


In [ ]:
%%time
test_bottleneck_features = model.predict_generator(test, test.samples // 7, verbose=1)

  94/1879 [>.............................] - ETA: 321s

In [ ]:
preds = mlp_vgg.predict(test_bottleneck_features,batch_size=7,verbose=1)

In [ ]:
preds_dict = dict()
for idx, img in enumerate(test.filenames):
    name = img.split('/')[1]
    tmp = dict()
    for i, p in enumerate(preds[idx]):
        tmp[i] = p
    preds_dict[name] = tmp

In [ ]:
submission = pd.DataFrame.from_dict(preds_dict,orient='index')
submission.columns = sorted(list(train_bottleneck.class_indices.keys()))
submission.reset_index(level=0, inplace=True)
submission.rename(columns={'index':'image'},inplace=True)

submission.to_csv('./submission.csv',index=False)